In [1]:
!pip install yfinance pandas numpy matplotlib seaborn scipy ipywidgets

import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display
from scipy.optimize import minimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.3 MB/s eta 0:00:00


In [2]:
def get_financials(ticker, start_date, end_date):
    stock = yf.Ticker(ticker)
    income_stmt = stock.financials
    balance_sheet = stock.balance_sheet
    cashflow_stmt = stock.cashflow
    stock_history = stock.history(start=start_date, end=end_date)

    return stock, income_stmt, balance_sheet, cashflow_stmt, stock_history

def get_free_cash_flows(cashflow_stmt):
    fcf = cashflow_stmt.loc['Free Cash Flow']
    fcf.ffill(inplace=True)
    return fcf[::-1]

def calculate_cagr(revenue):
    revenue = revenue.dropna()
    if len(revenue) < 2:
        return np.nan

    first_value = revenue.iloc[0]
    last_value = revenue.iloc[-1]

    if first_value <= 0:
        return np.nan

    cagr = ((last_value / first_value) ** (1 / (len(revenue) - 1))) - 1
    return cagr

def calculate_wacc(stock, balance_sheet):
    risk_free_rate = 0.043
    market_return = 0.11

    beta = stock.info['beta']
    cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)

    total_debt = balance_sheet.loc['Long Term Debt'].sum()
    total_equity = balance_sheet.loc['Ordinary Shares Number'].sum() * stock.history(period="1d")['Close'].iloc[-1]

    cost_of_debt = 0.043
    tax_rate = 0.21

    wacc = (total_equity / (total_equity + total_debt)) * cost_of_equity + (total_debt / (total_equity + total_debt)) * cost_of_debt * (1 - tax_rate)
    return wacc

def project_cash_flows(fcf, g1=0.20, g2=0.04, years_growth=10, years_terminal=10):
    projected_growth_fcfs = [fcf * (1 + g1) ** i for i in range(1, years_growth + 1)]
    last_growth_fcf = projected_growth_fcfs[-1]
    projected_terminal_fcfs = [last_growth_fcf * (1 + g2) ** i for i in range(1, years_terminal + 1)]

    return projected_growth_fcfs, projected_terminal_fcfs

def calculate_terminal_value(last_fcf, wacc, g2=0.04):
    return (last_fcf * (1 + g2)) / (wacc - g2)

def discount_cash_flows(growth_fcfs, terminal_fcfs, wacc):
    growth_fcfs = np.array(growth_fcfs, dtype=float)
    terminal_fcfs = np.array(terminal_fcfs, dtype=float)

    discounted_growth_fcfs = np.array([fcf / (1 + wacc) ** (i + 1) for i, fcf in enumerate(growth_fcfs)])

    last_growth_year = len(growth_fcfs)
    discounted_terminal_fcfs = np.array([
        fcf / (1 + wacc) ** (last_growth_year + i + 1) for i, fcf in enumerate(terminal_fcfs)
    ])

    return float(np.sum(discounted_growth_fcfs) + np.sum(discounted_terminal_fcfs))



#Here, I computed the main Discounted Cash Flow Model
def dcf_model(ticker, start_date, end_date):
    stock, income_stmt, balance_sheet, cashflow_stmt, stock_history = get_financials(ticker, start_date, end_date)

    fcf = get_free_cash_flows(cashflow_stmt)

    revenue = income_stmt.loc['Total Revenue']
    cagr = calculate_cagr(revenue)

    wacc = calculate_wacc(stock, balance_sheet)

    growth_fcfs, terminal_fcfs = project_cash_flows(fcf, g1=0.20, g2=0.04)

    terminal_value = calculate_terminal_value(terminal_fcfs[-1], wacc)
    discounted_terminal_value = terminal_value / (1 + wacc) ** 20

    intrinsic_enterprise_value = discount_cash_flows(growth_fcfs, terminal_fcfs, wacc) + discounted_terminal_value

    if isinstance(intrinsic_enterprise_value, pd.Series):
      intrinsic_enterprise_value = intrinsic_enterprise_value.iloc[0]
    else:
      intrinsic_enterprise_value = float(intrinsic_enterprise_value)

    current_stock_price = stock.history(period='1d')['Close'].iloc[-1]

    if isinstance(current_stock_price, pd.Series):
      current_stock_price = current_stock_price.iloc[0]
    else:
      current_stock_price = float(current_stock_price)

    num_shares = stock.info['sharesOutstanding']

    intrinsic_price = intrinsic_enterprise_value / num_shares

    print(f"\n📊 DCF Valuation for {ticker}")
    print(f"✅ Enterprise Value (Intrinsic): ${intrinsic_enterprise_value:.2f}")
    print(f"✅ Intrinsic Value Per Share: ${intrinsic_price:.2f}")
    print(f"📌 Current Stock Price: ${current_stock_price:.2f}")


ticker_input = widgets.Text(description="Ticker:")
start_date_input = widgets.DatePicker(description="Start Date:")
end_date_input = widgets.DatePicker(description="End Date:")
compute_button = widgets.Button(description="Compute DCF", button_style='success')

def on_compute_button_click(b):
    ticker = ticker_input.value.upper()
    start_date = start_date_input.value.strftime('%Y-%m-%d')
    end_date = end_date_input.value.strftime('%Y-%m-%d')

    if ticker and start_date and end_date:
        dcf_model(ticker, start_date, end_date)
    else:
        print("Please enter a valid stock ticker, start date, and end date.")

compute_button.on_click(on_compute_button_click)

ui = widgets.VBox([
    widgets.HBox([ticker_input, start_date_input, end_date_input]),
    compute_button
])

display(ui)

<ipython-input-2-cc0b486f0baf>:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fcf.ffill(inplace=True)



📊 DCF Valuation for NVDA
✅ Enterprise Value (Intrinsic): $2196893390137.08
✅ Intrinsic Value Per Share: $90.04
📌 Current Stock Price: $121.67
